In [1]:
import cbpro
import pandas as pd

from api import Coinbase_API
from api import Coinbase_secret
from api import Coinbase_pass

api_key = Coinbase_API
api_secret = Coinbase_secret
api_pass = Coinbase_pass

from coinbase.wallet.client import Client

In [9]:
help(cbpro.WebsocketClient)

Help on class WebsocketClient in module cbpro.websocket_client:

class WebsocketClient(builtins.object)
 |  WebsocketClient(url='wss://ws-feed.pro.coinbase.com', products=None, message_type='subscribe', mongo_collection=None, should_print=True, auth=False, api_key='', api_secret='', api_passphrase='', channels=None)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, url='wss://ws-feed.pro.coinbase.com', products=None, message_type='subscribe', mongo_collection=None, should_print=True, auth=False, api_key='', api_secret='', api_passphrase='', channels=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  close(self)
 |  
 |  on_close(self)
 |  
 |  on_error(self, e, data=None)
 |  
 |  on_message(self, msg)
 |  
 |  on_open(self)
 |  
 |  start(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakr

In [10]:
help(client)

Help on AuthenticatedClient in module cbpro.authenticated_client object:

class AuthenticatedClient(cbpro.public_client.PublicClient)
 |  AuthenticatedClient(key, b64secret, passphrase, api_url='https://api.pro.coinbase.com')
 |  
 |  Provides access to Private Endpoints on the cbpro API.
 |  
 |  All requests default to the live `api_url`: 'https://api.pro.coinbase.com'.
 |  To test your application using the sandbox modify the `api_url`.
 |  
 |  Attributes:
 |      url (str): The api url for this client instance to use.
 |      auth (CBProAuth): Custom authentication handler for each request.
 |      session (requests.Session): Persistent HTTP connection object.
 |  
 |  Method resolution order:
 |      AuthenticatedClient
 |      cbpro.public_client.PublicClient
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, key, b64secret, passphrase, api_url='https://api.pro.coinbase.com')
 |      Create an instance of the AuthenticatedClient class.
 |      
 |    

In [2]:
url = 'https://api.pro.coinbase.com'
client = cbpro.AuthenticatedClient(
    api_key,
    api_secret,
    api_pass,
    api_url = url)

total = 0
message = []

accounts = client.get_accounts()
for wallet in accounts:
    if wallet['balance'] > '0.0000000000000000':
        message.append(str(wallet['currency']) + ' ' + str(round(float(wallet['balance']),4)))
    value = str(wallet['balance']).replace('USD','')
    total += round(float(value), 4)
#message.append('Total Balance: ' + 'USD ' + str(total))
print ('\n'.join( message ))
print('Total Balance: ' + 'USD ' + str(total))

ADA 0.01
AVAX 0.0007
DASH 0.0002
LINK 0.0
USD 0.0067
USDC 0.0
USDT 0.0029
ZEC 31.4006
Total Balance: USD 31.4211


In [3]:
class TextWebsocketClient(cbpro.WebsocketClient):
    def on_open(self):  
        self.url = 'wss://ws-feed.prime.coinbase.com'
        self.message_count = 0
    
    def on_message(self,msg):
        self.message_count =+ 1
        msg_type = msg.get('type', None)
        if msg_type == 'ticker':
            time_val = msg.get('time',('-'*27))
            price_val = msg.get('price', None)
            price_val = float(price_val) if price_val is not None else 'None'
            product_id = msg.get('product_id', None)
            
            print(f'{time_val:30} {price_val:3f} {product_id}\tchannel type:{msg_type}')
    
    def on_close(self):
        print(f'<---Websocket connection closed--->\n\tTotal messages: {self.message_count}')

In [4]:
stream = TextWebsocketClient(products=['ETH-USD'],channels=['ticker'])

In [5]:
help(stream)

Help on TextWebsocketClient in module __main__ object:

class TextWebsocketClient(cbpro.websocket_client.WebsocketClient)
 |  TextWebsocketClient(url='wss://ws-feed.pro.coinbase.com', products=None, message_type='subscribe', mongo_collection=None, should_print=True, auth=False, api_key='', api_secret='', api_passphrase='', channels=None)
 |  
 |  Method resolution order:
 |      TextWebsocketClient
 |      cbpro.websocket_client.WebsocketClient
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  on_close(self)
 |  
 |  on_message(self, msg)
 |  
 |  on_open(self)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from cbpro.websocket_client.WebsocketClient:
 |  
 |  __init__(self, url='wss://ws-feed.pro.coinbase.com', products=None, message_type='subscribe', mongo_collection=None, should_print=True, auth=False, api_key='', api_secret='', api_passphrase='', channels=None)
 |      Initialize self.  See help(type(self)) for ac

Help on class WebsocketClient in module cbpro.websocket_client:

class WebsocketClient(builtins.object)
 |  WebsocketClient(url='wss://ws-feed.pro.coinbase.com', products=None, message_type='subscribe', mongo_collection=None, should_print=True, auth=False, api_key='', api_secret='', api_passphrase='', channels=None)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, url='wss://ws-feed.pro.coinbase.com', products=None, message_type='subscribe', mongo_collection=None, should_print=True, auth=False, api_key='', api_secret='', api_passphrase='', channels=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  close(self)
 |  
 |  on_close(self)
 |  
 |  on_error(self, e, data=None)
 |  
 |  on_message(self, msg)
 |  
 |  on_open(self)
 |  
 |  start(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakr

### Authenticated Client

In [ ]:
from api import Coinbase_API
from api import Coinbase_secret
from api import Coinbase_pass

api_key = Coinbase_API
api_secret = Coinbase_secret
api_pass = Coinbase_pass

In [ ]:
url = 'https://api.pro.coinbase.com'
client = cbpro.AuthenticatedClient(
    api_key,
    api_secret,
    api_pass,
    api_url = url)

In [34]:
accounts = client.get_accounts()

In [57]:
for acc in accounts:
    currency = acc.get('currency')
    if currency == 'SHIB':
        acc_id = acc.get('id')

In [58]:
acc_history = client.get_account_history(acc_id)

In [60]:
import json

for hist in acc_history:
    print(json.dumps(hist, indent=1))
    